# Задача
Необходимо создать модель, которая будет предсказывать стоимость автомобиля по его характеристикам. Для оценки использовать метрику MAPE 

В данном ноутбуке мы собрали в датасет все спарсенные файлы, провели разведывательный анализ данных и кодировку. На выходе получили готовый для обучения датасет.

Также в этом проекте мы использовали:
* Ноутбук, через который парсили https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parsers-data
* Спарсенный датасет https://www.kaggle.com/juliadeinego/data-car-sales
* Ноутбук, в котором провели EDA https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parser-eda
* Ноутбук, в котором провели обучение https://www.kaggle.com/juliadeinego/sf-dst-car-price-prediction-super-parsers-ml


# Загрузка и предподготовка данных

In [ ]:
#Импорт библиотек
import os
import numpy as np 
import pandas as pd 
import pandas_profiling
from pandas import Series
from itertools import combinations
from scipy.stats import ttest_ind

import re
import datetime

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.feature_selection import mutual_info_classif, f_classif

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
#Импорт данных из соревнования

data_test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv')
data_sample = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')


#Импорт своих данных

data_train = pd.DataFrame()
for dirname, _, filenames in os.walk('../input/data-car-sales'):
    
    for filename in filenames:
        try:
            df = pd.read_csv(os.path.join(dirname, filename))        
            data_train = data_train.append(df, ignore_index = True)
        except:
            df = pd.read_csv(os.path.join(dirname, filename),sep=';')        
            data_train = data_train.append(df, ignore_index = True)
            
print(len(data_train))

Удалим дубликаты и пустые строки из спарсенных данных

In [ ]:
data_train = data_train.drop_duplicates(subset=['sell_id'])
data_train = data_train.dropna(how='all')
len(data_train)

In [ ]:
#посмотрим какие колонки из data_train есть в data_test

#создадим пустой список, в который добавим колонки data_train, присутствующие в data_test
in_test = []
#создадим пустой список, в который добавим колонки data_train, отсутствующие в data_test
not_in_test = []

for column in data_train.columns:
    if column in data_test.columns:
        in_test.append(column)
    else:
        not_in_test.append(column)

Посмотрим на колонки data_train, присутствующие в data_test

In [ ]:
in_test

Посмотрим на колонки data_train, отсутствующие в data_test¶

In [ ]:
not_in_test

Удалим из data_train колонки, отсутствующие в data_test, кроме price

In [ ]:
columns_to_drop = not_in_test[:4] + not_in_test[5:]

In [ ]:
data_train = data_train.drop(columns_to_drop, axis = 1)
data_train.head(3)

Перезапишем data_test с теми колонками, которые есть и в data_train, и в data_test

In [ ]:
data_test = data_test[in_test]
data_test.head(3)

Подготовим итоговый датасет для EDA

In [ ]:
#приведем целевую переменную к числовому типу
data_train['price'] = data_train.price.apply(lambda x: int("".join(filter(str.isdigit, x))))

#объединяем трейн и тест, для учета всех возможных значений. Помечаем где у нас трейн
data_train['sample_'] = 0
data_test['sample_'] = 1
data_full = pd.concat([data_test, data_train])
data_full = data_full.reset_index().drop(['index'], axis = 1)

#задаем порядок столбцов
data_full = data_full[['parsing_unixtime', 'sell_id', 'car_url', 'description',
       'image', 'bodyType', 'color', 'engineDisplacement', 'enginePower',
       'fuelType', 'mileage', 'productionDate', 'vehicleTransmission',
       'Владельцы', 'ПТС', 'Привод', 'Руль', 'Состояние', 'Таможня',
       'equipment_dict', 'brand', 'model_name', 'name', 'sample_', 'price']]

Удаляем дубликаты по ссылкам

In [ ]:
data_full = data_full.drop_duplicates(subset=['car_url'])
len(data_full)

Проверим на дубликаты по ИД объявлениям. Для этого приведем их к одному виду int, т.к. формат в трейне и тесте отличается. 

In [ ]:
data_full['sell_id'] = data_full.sell_id.apply(lambda x: int(str(x).replace('№ ','')))
data_full['sell_id'].value_counts()

Обнаружены дублирующиеся ИД, посмотрим чем отличаются

In [ ]:
data_full[data_full['sell_id'] == 1100086706]

У объявлений разные ссылки с идентичными параметрами.

https://auto.ru/cars/used/sale/toyota/allex/1100086706-8d67a4c7/

https://auto.ru/cars/used/sale/toyota/corolla/1100086706-8d67a4c7/

Пройдя по ссылками убедились, что данные спарсены корректно, и на самом деле существует два идентичных объявления. Подобные объявления будем оставлять только в одном экземпляре.

In [ ]:
data_full = data_full.drop_duplicates(subset=['sell_id'])
len(data_full)

# Описание датасета

* parsing_unixtime - дата время парсинга 
* sell_id - идентификатор объявления
* car_url - ссылка на объявление
* description - описание, комментарий продавца
* image - фото автомобиля
* bodyType - кузов
* color - цвет
* engineDisplacement - объем двигателя
* enginePower - мощность двигателя
* fuelType - тип двигателя (Бензин, Дизель, Гибрид, электро, Газ, Бензин+Газ, Дизель+Газ, Гибрид+Газ
* mileage - пробег
* productionDate - год выпуска автомобиля
* vehicleTransmission - тип коробки передач (механическая, автоматическая, вариатор, роботизированная)
* Владельцы - число владельцев авто (1, 2, 3 и более)
* ПТС - оригинальность ПТС (Оригинал, Дубликат)
* Привод - тип привода (передний, полный, задний)
* Руль - размещение рулевого колеса (Левый, Правый)
* Состояние - указывает на необходимость ремонта (Не требует ремонта, Битый / не на ходу)
* Таможня - необходимость процедуры растаможивания
* equipment_dict - дополнительная информация об автомобиле
* brand - марка авто (lada (ваз), toyota, kia, nissan, hyundai, bmw, mercedes-benz, renault, ford,
            skoda, mitsubishi, audi, opel, mazda, honda, volkswagen, lexus, peugeot, volvo,
            land rover, infiniti, subaru, chery, suzuki, citroen, lifan, geely, porsche, газ, haval)
* model_name - название модели
* name - расширенные технические данные
* sample_ - принадлежность к тестовому датасету
* price - цена авто

# EDA
Подготовим списки для распределения признаков, которые будем использовать в обучении и посмотрим на признаки ближе

In [ ]:
#распределяем признаки по типу: бинарные (признаки с 2мя уникальными значениями), категориальные (более 2х уникальных значений) и числовые 
bin_cols = [] 
cat_cols = [] 
num_cols = [] 

Выгрузим предварительный отчет в html

In [ ]:
car_sales_report = pandas_profiling.ProfileReport(data_full)
car_sales_report.to_file("car_sales_report.html")

Посмотрим содержание итогового датасета

In [ ]:
display(data_full.info())
display(data_full.sample(3))

## 0. parsing_unixtime - дата время парсинга
Преобразуем в удобный тип и посмотрим на распределение по датам

In [ ]:
data_full['datetime'] = data_full['parsing_unixtime'].apply(lambda x: datetime.datetime.fromtimestamp(x))
data_full['datetime'].dt.date.value_counts()

Судя по всему парсинг был запущен дважды. Тестовые данные собирались с 19 по 26 октября 2020, и данные для обучения с 15 по 16 апреля 2021.

## 1. sell_id - идентификатор объявления
Из html отчета car_sales_report видим, данный признак уникален, пропусков не осталось. Признак является идентификатором объявления. Смысловой нагрузки не несет, для обучение не отбираем

## 2. car_url - ссылка на объявление
По html отчету car_sales_report видим аналогичную с sell_id картину. Признак не будем использовать в обучении.

## 3. description - описание, комментарий продавца
Из html отчета car_sales_report видим - есть пропуски, много уникальных значений. Попробуем сгенерировать 2 признака на основании этого:
1 бинарный - наличие описания, 2 числовой - количество символов в описании.

In [ ]:
data_full['description_len'] = data_full.description.apply(lambda x: len(str(x)))
data_full['description_is'] = data_full['description'].isna()

num_cols.append('description_len')
bin_cols.append('description_is')

## 4. image - фото автомобиля
Из html отчета car_sales_report видим, что фото не уникальны, а это странно, т.к. продаются б/у автомобили, соответственно ожидаем, что фото будут индивидуальные. Посмотрим пару объявлений с одинаковыми фотографиями.

In [ ]:
#получаем самые частые фотографии
count_photos = data_full.image.value_counts()
display(count_photos.head(3))

#получаем ссылки на объявления, использующие самую популярную фотографию
data_full[data_full.image == count_photos.head(1).index[0]].car_url.values

Самая частая фотография используется в 1987 объявлениях. Пройдя по нескольким ссылкам видим, что продавец не загрузил фотографию, и используется стандартное фото из каталога.

Проверим пару ссылок с уникальными фотографиями.

In [ ]:
#получим уникальные фотографии
display(count_photos.sort_values().head(3))

#получим ссылки на объявления с этими фотографиями
data_full[data_full.image.isin(count_photos.sort_values().head(3).index)].car_url.values

Действительно, уникальные фотографии являются реальными. Наверняка покупатель сразу хочет видеть в каком состоянии авто. Сгенерируем для этого новый признак "real_photo". 
Скорее всего ссылки из каталога должны быть в одном формате. Попробуем зацепиться за фразу "get-verba" в ссылке.
Проверим верна ли догадка на парочке фотографий в единственном экземпляре, для которых проставилось значение о том, что фото не реальное

In [ ]:
#генерируем признак
data_full['real_photo'] = data_full.image.apply(lambda x: x.find('get-verba') == -1)

#получаем нереальные фотографии в единственном экземпляре
real_photos = data_full[data_full['real_photo'] == False].image.value_counts()
display(real_photos.sort_values().head(3))

#получаем ссылки на объявления
data_full[data_full.image.isin(real_photos.head(3).index)].car_url.values

Догадка подтвердилась. Реальность фотографии определяется верно. Используем новый признак для обучения. Определим его как бинарный.

In [ ]:
bin_cols.append('real_photo')

## 5. bodyType
Из html отчета car_sales_report видим, что данные заполнены полностью без пропусков, признак явно категориальный. Судя по распределению, некоторых видов не так много, и некоторые значения однотипные. Попробуем их немного сгруппировать в более общие типы. 

In [ ]:
def get_bodyClass(bodyType):
    
    bodyClass = 'other'
    
    dict_bodyTypes = {'седан' : ['седан', 'седан-хардтоп', 'фастбек'],
                          'внедорожник' : ['внедорожник 5 дв.', 'внедорожник 3 дв.', 'внедорожник открытый'],
                          'хэтчбек': ['хэтчбек 5 дв.', 'хэтчбек 4 дв.', 'хэтчбек 3 дв.'],
                          'универсал': ['универсал 5 дв.'],
                          'лифтбек': ['лифтбек'],
                          'автобус': ['микровэн', 'минивэн', 'компактвэн'],                      
                          'грузо-пассажирский': ['пикап полуторная кабина', 'пикап одинарная кабина', 'пикап двойная кабина', 'фургон']
                          #,'купе': ['купе', 'купе-хардтоп', 'седан 2 дв.'],
                          #,'открытый/съемный верх': ['кабриолет', 'фаэтон', 'тарга', 'родстер']
                     }
    
    for key, value in dict_bodyTypes.items():   
        if bodyType in value:
            bodyClass = key
            break
    
    return bodyClass

data_full['bodyClass'] = data_full.bodyType.apply(lambda x: get_bodyClass(x))
data_full.bodyClass.value_counts()

Удалось сократить с 26 до 8 типов кузовов. Данные распределились более равномерно, единичные экземпляры объединились в группу other. Признак отнесем к категориальным

In [ ]:
cat_cols.append('bodyClass')

## 6. color - цвет
Из html отчета car_sales_report видим, что данный признак лучше отнести к категориальному. Значения заполнены хорошо, пропусков нет. Уникальных значений 16 без дублирующихся по смыслу. Оставим в таком виде и добавим в список категориальных

In [ ]:
cat_cols.append('color')

## 7. engineDisplacement

- объём двигателя, л; тип данных - object

In [ ]:
data_full.engineDisplacement.value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.engineDisplacement.isna().sum()

Посмотрим все уникальные значения признака.

In [ ]:
data_full.engineDisplacement.unique()

Заменим значение ' LTR' на 0, так как здесь отсутствует ДВС.

In [ ]:
data_full.engineDisplacement = data_full.engineDisplacement.apply(lambda x: '0.0' if x == ' LTR' else x)

Уберем из этого списка одно неподходящее для преобразования в число значение 'Hyundai Grand Starex I'.

In [ ]:
data_full = data_full.drop(data_full[data_full.engineDisplacement == 'Hyundai Grand Starex I'].index)

Преобразуем признак из object в числовой.

In [ ]:
pattern = re.compile('\d+\.\d+')

data_full['engineDisplacement'] = data_full['engineDisplacement'].apply(lambda x: float(pattern.findall(x)[0]))

Построим гистограмму распределения признака.

In [ ]:
data_full['engineDisplacement'].hist()

В результате пребразований получен числовой признак. Распределение можно считать нормальным.

In [ ]:
num_cols.append('engineDisplacement')

## 8. enginePower

- мощность двигателя, л. с.; тип данных - object

In [ ]:
data_full.enginePower.value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.enginePower.isna().sum()

Посмотрим все уникальные значения признака.

In [ ]:
data_full.enginePower.unique()

Преобразуем признак из object в числовой.

In [ ]:
pattern = re.compile('\d+')

data_full['enginePower'] = data_full['enginePower'].apply(lambda x: float(pattern.findall(x)[0]))

Построим гистограмму распределения признака.

In [ ]:
data_full['enginePower'].hist()

В результате пребразований получен числовой признак. Распределение можно считать нормальным.

In [ ]:
num_cols.append('enginePower')

## 9. fuelType

- тип топлива; тип данных - object

In [ ]:
data_full.fuelType.value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.fuelType.isna().sum()

Посмотрим все уникальные значения.

In [ ]:
data_full.fuelType.unique()

Преобразуем ' Газ, газобаллонное оборудование' в ' Газ'.

In [ ]:
data_full.fuelType = data_full.fuelType.apply(lambda x: 'газ' if x == ' Газ, газобаллонное оборудование' else x)

Удалим одну строку со значением '1618546594.0'.

In [ ]:
data_full = data_full.drop(data_full[data_full.fuelType == '1618546594.0'].index)

Приведем все значения к нижнему регистру.

In [ ]:
data_full.fuelType = data_full.fuelType.apply(lambda x: x.lower())

Приведем уникальные значения к единообразному виду.

In [ ]:
pattern = re.compile('[а-я]+')

data_full.fuelType = data_full.fuelType.apply(lambda x: 'гибрид' if len(x.split(',')) == 2 else pattern.findall(x)[0])

In [ ]:
data_full.fuelType.value_counts().plot.barh()

В результате преобразований получен категориальный признак.

In [ ]:
cat_cols.append('fuelType')

## 10. mileage

- пробег автомобиля, км; тип данных - object

In [ ]:
data_full.mileage.value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.mileage.isna().sum()

Посмотрим на структуру значений.

In [ ]:
data_full.mileage[0]

In [ ]:
data_full.mileage[237229]

Уберем части строки равные '\xa0' из всех значений признака.

In [ ]:
data_full['mileage'] = data_full['mileage'].apply(lambda x: str(x).replace('\xa0', ''))

Преобразуем признак из object в числовой.

In [ ]:
pattern = re.compile('\d+')

data_full['mileage'] = data_full['mileage'].apply(lambda x: int(pattern.findall(x)[0]))

Построим гистограмму распределения признака.

In [ ]:
data_full['mileage'].hist()

В результате пребразований получен числовой признак. Распределение можно считать нормальным.

In [ ]:
num_cols.append('mileage')

## 11. productionDate

- год производства автомобиля; тип данных - float

In [ ]:
data_full.productionDate.value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.productionDate.isna().sum()

Посмотрим на уникальные значения.

In [ ]:
data_full.productionDate.unique()

Преобразуем признак из object в числовой.

In [ ]:
pattern = re.compile('\d+')

data_full['productionDate'] = data_full['productionDate'].apply(lambda x: int(pattern.findall(str(x))[0]))

Построим гистограмму распределения признака.

In [ ]:
data_full['productionDate'].hist()

В результате пребразований получен числовой признак. На распределении визуально заметны выбросы, но они не противоречат здравому смыслу, поэтому их можно оставить.

In [ ]:
num_cols.append('productionDate')

## 12. vehicleTransmission

- тип трансмиссии автомобиля; тип данных - object

In [ ]:
data_full.vehicleTransmission.value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.productionDate.isna().sum()

In [ ]:
data_full['vehicleTransmission'].value_counts().plot.barh()

В данном признаке дополнительных преобразований не требуется. Признак является категориальным.

In [ ]:
cat_cols.append('vehicleTransmission')

## 13. Владельцы

- количество владельцев автомобиля; тип данных - object

In [ ]:
data_full['Владельцы'].value_counts()

Проверим признак на пустые ячейки.

In [ ]:
data_full.productionDate.isna().sum()

Посмотрим на уникальные значения.

In [ ]:
data_full['Владельцы'].unique()

В данном столбце мы имеем категориальные значения. Преобразуем данные к презентабельному виду - заменим части строки равные '\xa0' на ' ' во всех значениях признака.

In [ ]:
data_full['Владельцы'] = data_full['Владельцы'].apply(lambda x: str(x).replace('\xa0', ' '))

In [ ]:
data_full['Владельцы'].value_counts().plot.barh()

In [ ]:
cat_cols.append('Владельцы')

## 14. ПТС

- вид технического паспорта автомобиля; тип данных - object

В отчете html видим есть 1 пропуск, заполним его самым частым значением

In [ ]:
data_full['ПТС'] = data_full['ПТС'].fillna(data_full['ПТС'].value_counts().head(1).index[0])

In [ ]:
data_full['ПТС'].value_counts()

In [ ]:
sns.countplot(x = 'ПТС', data = data_full)

Видно, что оригинальных ПТС намного больше, чем дубликатов.

Здесь мы имеем бинарный признак. Дополнительных преобразований не требуется.

In [ ]:
bin_cols.append('ПТС')

## 15. Привод

- вид привода автомобиля, тип данных - object

In [ ]:
data_full['Привод'].value_counts()

Проверим признак на ниличие пустых значений

In [ ]:
data_full.Привод.isna().sum()

In [ ]:
sns.countplot(x='Привод', data=data_full)

Признак категориальный. Дополнительная обработка не требуется

In [ ]:
cat_cols.append('Привод')

## 16. Руль

расположение рулевого колеса; тип данных - object

In [ ]:
data_full['Руль'].value_counts()

Проверим наличие пустых значений

In [ ]:
data_full.Руль.isna().sum()

In [ ]:
sns.countplot(x='Руль', data=data_full)

Признак бинарный. Дополнительная обработка не требуется

In [ ]:
bin_cols.append('Руль')

## 17. Состояние

указатель необходимости ремонта автомобиля; тип данных - object

In [ ]:
data_full['Состояние'].value_counts()

Проверим наличие пустых значений

In [ ]:
data_full.Состояние.isna().sum()

In [ ]:
sns.countplot(x='Состояние', data=data_full)

Признак бинарный. Дополнительная обработка не требуется

In [ ]:
bin_cols.append('Состояние')

## 18. Таможня
Признак одинаков для всех строк и не несет полезной информации. В модель не включаем.

## 19 equipment_dict
Данные этого поля дублируются в других полях датасета. Полезной информации не несет. В модель не включаем.

## 20. brand
марка автомобиля; тип данных - object

In [ ]:
data_full['brand'].value_counts()

Имеются повторы. Обработаем данные

In [ ]:
# приведем названия марок авто к нижнему регистру
data_full['brand'] = data_full['brand'].apply(lambda x: x.lower())

# заменим название mercedes-benz на mercedes
data_full['brand'] = data_full['brand'].apply(
    lambda x: x.replace('mercedes-benz', 'mercedes'))

Посмотрим результат

In [ ]:
data_full['brand'].value_counts()

Признак категориальный. Дополнительная обработка не требуется.

In [ ]:
cat_cols.append('brand')

## 21. model_name
модель автомобиля; тип данных - object

In [ ]:
data_full['model_name'].value_counts()

Обработаем данные

In [ ]:
# приведем значения к нижнему регистру и удажим лишние пробелы
data_full['model_name'] = data_full['model_name'].apply(
    lambda x: x.lower().strip())
data_full['model_name'].value_counts()

Число уникальных значений немного уменьшилось

Признак категориальный.

In [ ]:
cat_cols.append('model_name')

## 22. name
полная информация о двигателе; тип данных - object

Информация в данном поле дублирует информацию в других полях. В модель не включаем.

## 23. sample_
признак включения в тестовый датасет

## 24. price

- стоимость автомобиля. тип данных - float

In [ ]:
sns.distplot(data_full[data_full['sample_']==0]['price']).get_figure()

На распределении визуально заметны выбросы, но они не противоречат здравому смыслу, поэтому их можно оставить.

## Оставим в датасете только отобранные признаки

In [ ]:
data_full = data_full[cat_cols + bin_cols + num_cols + ['price', 'sample_']]

# Обработка и анализ числовых признаков

Посмотрим на все числовые признаки.

In [ ]:
data_full[data_full['sample_'] == 0][num_cols + ['price']]

Постороим матрицу корреляций числовых переменных друг с другом и с целевой переменной price.

In [ ]:
plt.figure(figsize=(12,7))
sns.heatmap(data_full[data_full['sample_'] == 0][num_cols + ['price']].corr(), annot=True)

Здесь мы видим, что сильную корреляцию между собой имеют признаки engineDisplacement (объем двигателя) и enginePower (мощность двигателя) - 0.88. Поэтому удаляем один признак - engineDisplacement (объем двигателя), т.к. enginePower (мощность двигателя) более презентабелен.

In [ ]:
data_full = data_full.drop(['engineDisplacement'], axis = 1)

Также удалим его из списка числовых переменных num_cols.

In [ ]:
num_cols = num_cols[:1] + num_cols[2:]

Наблюдается достаточно высокая корреляция между productionDate (год производства автомобиля) и mileage (пробег автомобиля) - 0.66. Но здесь можно оставить оба признака.

Корреляция среди обучающих признаков с целевой переменной price (цена автомобиля) самая высокая у enginePower (мощность двигателя) - 0.65, но мы его оставляем.

Оценим значимость числовых переменных.

In [ ]:
imp_num = Series(f_classif(data_full[num_cols][data_full['sample_'] == 0], data_full[data_full['sample_'] == 0]['price'])[0], 
                 index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh')

Из графика видим. что наиболее важный признак для предсказания - это enginePower (мощность двигателя), а наименее - mileage (пробег автомобиля).

# Обработка и анализ бинарных признаков
Преобразуем отобранные бинарные признаки

In [ ]:
label_encoder = LabelEncoder()
for column in bin_cols:
    data_full[column] = label_encoder.fit_transform(data_full[column])

Проверим, что все корректно преобразовалось

In [ ]:
data_full[bin_cols].head(5)

In [ ]:
for i in bin_cols:
    display(data_full[i].value_counts())

Посмотрим значимость

In [ ]:
imp_cat = Series(mutual_info_classif(data_full[data_full.sample_ == 0][bin_cols], data_full[data_full.sample_ == 0]['price'],
                                     discrete_features =True), index = bin_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

На цену меньше всего влияет состояние, что противоречит здравому смыслу... Видимо сказывается несбалансированность этого признака, почти все объявления с одним значением. А самый больший вклад в цену вносит наличие реальной фотографии.

# Обработка и анализ категориальных признаков

In [ ]:
#преобразование, значимость, выводы. Label encoding
data_full[cat_cols]

Сделаем копию датасета

In [ ]:
df = data_full.copy()

Произведем Label-encoding категориальных переменных

In [ ]:
label_encoder = LabelEncoder()
for column in cat_cols:
    df[column] = label_encoder.fit_transform(df[column])

Посмотрим значимость категориальных переменных

In [ ]:
imp_cat = Series(mutual_info_classif(df[data_full.sample_ == 0][cat_cols], df[data_full.sample_ == 0]['price'],
                                     discrete_features =True), index = cat_cols)
imp_cat.sort_values(inplace = True)
imp_cat.plot(kind = 'barh')

Как видно из графиков, наибольшее влияние на целевую переменную оказывают марка и модель автомобиля. Большое влияние модели автомобиля обусловлено относительно небольшим числом записей для каждой модели.

Проверим, есть ли статистическая разница в распределении оценок по номинативным признакам, с помощью теста Стьюдента. Проверим нулевую гипотезу о том, что распределения цены по различным параметрам неразличимы

In [ ]:
def get_stat_dif(column):
    cols = df.loc[:, column].value_counts()
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        if ttest_ind(df.loc[df.loc[:, column] == comb[0], 'price'], 
                        df.loc[df.loc[:, column] == comb[1], 'price']).pvalue \
            <= 0.05/len(combinations_all): # Учли поправку Бонферони
            print('Найдены статистически значимые различия для колонки', column)
            break
            
for col in cat_cols:
    get_stat_dif(col)

Так как число уникальных значений для поля model_name очень большое не включаем его в модель

In [ ]:
cat_cols.remove('model_name')
data_full.drop(columns=['model_name'], inplace=True)

Создадим дамми переменные

In [ ]:
data_full = pd.get_dummies(data_full, columns = cat_cols)

In [ ]:
first_index_cat_cols = data_full.columns.to_list().index('sample_')+1
cat_cols = data_full.columns[first_index_cat_cols:].to_list()

Выгрузим данные в csv для дальнейшего обучения

In [ ]:
data_full.to_csv('data_full_EDA.csv', index = False)

Выгрузим названия колонок по типам

In [ ]:
df_columns = pd.DataFrame()

for col in num_cols:
    new_row = {'column_type': 'num', 'column_name': col} 
    df_columns = df_columns.append(new_row, ignore_index=True)

for col in bin_cols:
    new_row = {'column_type': 'bin', 'column_name': col} 
    df_columns = df_columns.append(new_row, ignore_index=True)

for col in cat_cols:
    new_row = {'column_type': 'cat', 'column_name': col} 
    df_columns = df_columns.append(new_row, ignore_index=True)
    
df_columns.to_csv('data_full_columns.csv', index = False)

In [ ]:
df = pd.read_csv('data_full_columns.csv')
data_full[df[df.column_type == 'cat'].column_name.values]

# Выводы
Спарсилось очень много дублей. Спарсенные данные не везде совпадают с тестовым датасетом, пришлось делать дополнительную обработку.
В данных очень много категориальных признаков. Некоторые признаки несбалансированы.

# Что можно улучшить:
1. Тут большое поле для feature engineering, много текста из которого можно вытащить доп информацию.  
2. Задействованы далеко не все из спарсенного, т.к. некоторые спарсенные данные не совпадают с тестовыми. Можно попытаться вытащить общую информацию из того что есть, или проанализировать и подкорректировать парсинг